In [31]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [32]:
def dfdx(x,f):
    y = f[0]
    z = f[1]
    
    dydx = np.zeros_like(f)
    dydx[0] = z
    dydx[1] = -1*y
    
    return dydx

In [33]:
def cash_karp_core_mv(x_i,y_i,nv,h,f):
    
    ni = 7
    nj = 6
    ci = np.zeros(ni)
    aij = np.zeros( (ni,nj) )
    bi = np.zeros(ni)
    bis = np.zeros(ni)
    
    ci[2] = 1./5.
    ci[3] = 3./10.
    ci[4] = 3./5.
    ci[5] = 1.
    ci[6] = 7./8.
    
    
    aij[2,1] = 1./5.
    aij[3,1] = 3./40.
    aij[4,1] = 3./10.
    aij[5,1] = -11./5.
    aij[6,1] = 1631./55296.
    
    
    aij[3,2] = 9./40.
    aij[4,2] = -9./10.
    aij[5,2] = 5./2.
    aij[6,2] = 175./512.
    
    
    aij[4,3] = 6./5.
    aij[5,3] = -70./27.
    aij[6,3] = 575./13824.
    
    aij[5,4] = 35./27.
    aij[6,4] = 44275./110592.
    
    aij[6,5] = 253./4096.
    
    
    
   
    bi[1] = 37./378.
    bi[2] = 0.
    bi[3] = 250./621.
    bi[4] = 125./594.
    bi[5] = 0.0
    bi[6] = 512./1771.
    
    
    bis[1] = 2825./27648.
    bis[2] = 0.0
    bis[3] = 18575./48384.
    bis[4] = 13525./55296.
    bis[5] = 277./14336.
    bis[6] = 1./4.
    
    ki = np.zeros((ni,nv))
    
    for i in range(1,ni):
        xn = x_i + ci[i]*h
        
        yn = y_i.copy()
        for j in range(1,i):
            yn += aij[i,j]*ki[j,:]
            
        ki[i,:] = h*f(xn, yn)
        
    ynpo = y_i.copy()
    ynpos = y_i.copy()
    
    for i in range(1,ni):
        ynpo += bi[i] *ki[i,:]
        ynpos += bis[i]*ki[i,:]
        
    delta = np.fabs(ynpo-ynpos)
    
    
    return ynpo, delta

In [34]:
def cash_karp_mv_ad(dfdx,x_i,y_i,nv,h,tol):
    
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    imax = 1000
    
    i = 0
    
    delta = np.full(nv,2*tol)
    
    h_step = h
    
    while(delta.max()/tol>1.0):
        
        y_ipo, delta = cash_karp_core_mv(x_i,y_i,nv,h_step,dfdx)
        
        if(delta.max()/tol>1.0):
            
            h_step *= SAFETY * (delta.max()/tol)**(-0.25)
            
        if(i>=imax):
            print("too many iterations in cash karp")
            raise StopIteration("Ending after i = ",i)
        
        
        i += 1
        
        
    h_new = np.fmin(h_step * (delta.max()/tol)**(-0.9), h_step * H_NEW_FAC)
    
    return y_ipo, h_new, h_step

In [35]:
def cash_karp_mv(dfdx,a,b,y_a,tol,verbose=False):
    
    xi = a
    yi = y_a.copy()
    
    h = 1.0e-4 * (b-a)
    
    imax = 1000
    
    i = 0
    
    nv = len(y_a)
    
    x = np.full(1,a)
    y = np.full( (1,nv), y_a)
    
    flag = True 
    
    while(flag):
        
        y_ipo, h_new, h_step = cash_karp_mv_ad(dfdx,xi,yi,nv,h,tol)
        
        h = h_new
        
        if(xi+h_step>b):
            
            h = b - xi
            
            y_ipo, h_new, h_step = cash_karp_mv_ad(dfdx,xi,yi,nv,h,tol)
            
            flag = False 
        
        xi += h_step
        yi = y_ipo.copy()
        
        x = np.append(x,xi)
        y_ipo = np.zeros((len(x),nv))
        y_ipo[0:len(x)-1,:] = y[:]
        y_ipo[-1,:] = yi[:]
        del y
        y = y_ipo
        
        
        if(i>=imax):
            print("max iterations reached")
            raise StopIteration("iteration number = ",i)

            
        i += 1
        
        if(verbose):
            s = "i = %3d\tx = %9.8f\ty = %9.8f\th = %9.8f\tb = %9.8f" % (i,xi,yi[0],h_step,b)
            print(s)
            
            
        if(xi==b):
            flag = False
            
    return x, y 



In [36]:


a = 0.0
b= 2.0*np.pi
y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance = 1.0e-6

x, y = cash_karp_mv(dfdx,a,b,y_0,tolerance,verbose=True)

plt.plot(x,y[:,0],'o',label='y(x)')
plt.plot(x,y[:,1],'o',label='z(x)')
plt.plot(x,np.sin(x),label='sin(x)')
plt.plot(x,np.cos(x),label='cos(x)')
plt.xlim([0,2*np.pi])
plt.ylim([-1.2,1.2])
plt.legend(frameon=False)

i =   1	x = 0.00062832	y = 0.00062832	h = 0.00062832	b = 6.28318531
i =   2	x = 0.00188496	y = 0.00188495	h = 0.00125664	b = 6.28318531
i =   3	x = 0.00439823	y = 0.00439822	h = 0.00251327	b = 6.28318531
i =   4	x = 0.00942478	y = 0.00942464	h = 0.00502655	b = 6.28318531
i =   5	x = 0.01403102	y = 0.01403056	h = 0.00460624	b = 6.28318531
i =   6	x = 0.01880112	y = 0.01880002	h = 0.00477010	b = 6.28318531
i =   7	x = 0.02350518	y = 0.02350303	h = 0.00470406	b = 6.28318531
i =   8	x = 0.02823582	y = 0.02823209	h = 0.00473064	b = 6.28318531
i =   9	x = 0.03295616	y = 0.03295022	h = 0.00472034	b = 6.28318531
i =  10	x = 0.03768104	y = 0.03767215	h = 0.00472488	b = 6.28318531
i =  11	x = 0.04240460	y = 0.04239192	h = 0.00472356	b = 6.28318531
i =  12	x = 0.04712925	y = 0.04711184	h = 0.00472465	b = 6.28318531
i =  13	x = 0.05185411	y = 0.05183091	h = 0.00472486	b = 6.28318531
i =  14	x = 0.05657960	y = 0.05654946	h = 0.00472549	b = 6.28318531
i =  15	x = 0.06130563	y = 0.06126729	h = 0.0047

StopIteration: ('iteration number = ', 1000)

In [ ]:



plt.plot(x, y[:,0]-np.sin(x), label="y(x) Error")
plt.plot(x, y[:,1]-np.cos(x), label="z(x) Error")
plt.legend()